In [6]:
import os
import pandas as pd
from PIL import Image

run_ichallenge_amd  = True
run_ichallenge_non_amd  = True
run_ichallenge_glaucoma = True
run_ichallenge_unlabelled = True



In [20]:
def generate_semisupseg_csv_file(images_path, masks_path, csv_path, weight, image_postfix=".jpg", mask_postfix=".bmp", split_percentage=90, new_mask_path=None):
    
    # have to call for each dataset that contains images + masks
    # finding image (.image_postfix) and mask (.mask_postfix) pairs for same base filename
    # this works if your_img_dir/A0001.jpg and your_mask_dir/A0001.bmp
    # optic nerve head
    # converts all the masks - 0 background, 255 foreground if new_mask_path is not None
    
    mode = []
    
    # need one file for each concept 
    image_path = list(set(os.listdir(images_path)) - {'desktop.ini'})
    image_id = [os.path.basename(x).replace(image_postfix, "") for x in image_path]
    image_path = [os.path.join(images_path, x) for x in image_path]
    a = {"image_path" : image_path, "id" : image_id}
    a = pd.DataFrame(data={"img_path" : image_path}, index=image_id)
    
    if masks_path is not None:
        mask_path  = list(set(os.listdir(masks_path)) - {'desktop.ini'})
        mask_id = [os.path.basename(x).replace(mask_postfix, "") for x in mask_path]
        mask_path = [os.path.join(masks_path, x) for x in mask_path]
        
        new_mask_list = []
        if new_mask_path is not None:
            
            for mp in mask_path:

                # all classes = 255 (will become 1), background = 0
                mask = Image.open(mp).convert('L')
                mask = mask.point( lambda p: 0 if p == 255 else 255 )
                mask = mask.convert('1')
                c = os.path.join(new_mask_path, os.path.basename(mp))
                new_mask_list.append(c)
                mask.save(c)
                #mask.show()
                #break
                #print("unique mask", np.unique(mask))
                #print(self.csv_data.iloc[index]['msk_path'])
                #print("unique mask", np.unique(np.asarray(mask)))
            mask_path = new_mask_list
                
        b = {"mask_path" : mask_path, "id" : mask_id}
        b = pd.DataFrame(data={"msk_path" : mask_path}, index=mask_id)
        csv_data = pd.concat([a, b], axis=1, join="inner")
        
    else:
        csv_data = a
        csv_data["mask_path"] = [None] * len(a.index)
        
    
    csv_data = csv_data.sample(frac=1, random_state=19)

    percent_train = int(len(csv_data)/100*split_percentage)
    percent_val = len(csv_data)-percent_train
    
    mode.extend(["train" for i in range(percent_train)])
    mode.extend(["val" for i in range(percent_val)])
    
    
    csv_data["mode"] = mode
    csv_data["weight"] = [weight]*len(mode)

    print(csv_data)
    
    csv_data.to_csv(csv_path, sep=';')
            

In [21]:
if run_ichallenge_amd:
    images_path =    r"C:\Users\Prinzessin\projects\image_data\iChallenge_AMD_OD_Fovea_lesions\images_AMD"
    masks_path  =    r"C:\Users\Prinzessin\projects\image_data\iChallenge_AMD_OD_Fovea_lesions\Disc_Masks"
    new_masks_path = r"C:\Users\Prinzessin\projects\image_data\iChallenge_AMD_OD_Fovea_lesions\Disc_Masks_bin"
    if not os.path.exists(new_masks_path):
        os.makedirs(new_masks_path)
    csv_path    = r"mt_data_ichallenge_amd.csv"
    weight = 1
    generate_semisupseg_csv_file(images_path, masks_path, csv_path, weight, image_postfix=".jpg", mask_postfix=".bmp", split_percentage=90, new_mask_path=new_masks_path)

                                                img_path  \
A0068  C:\Users\Prinzessin\projects\image_data\iChall...   
A0006  C:\Users\Prinzessin\projects\image_data\iChall...   
A0080  C:\Users\Prinzessin\projects\image_data\iChall...   
A0025  C:\Users\Prinzessin\projects\image_data\iChall...   
A0008  C:\Users\Prinzessin\projects\image_data\iChall...   
...                                                  ...   
A0030  C:\Users\Prinzessin\projects\image_data\iChall...   
A0081  C:\Users\Prinzessin\projects\image_data\iChall...   
A0034  C:\Users\Prinzessin\projects\image_data\iChall...   
A0004  C:\Users\Prinzessin\projects\image_data\iChall...   
A0087  C:\Users\Prinzessin\projects\image_data\iChall...   

                                                msk_path   mode  weight  
A0068  C:\Users\Prinzessin\projects\image_data\iChall...  train       1  
A0006  C:\Users\Prinzessin\projects\image_data\iChall...  train       1  
A0080  C:\Users\Prinzessin\projects\image_data\iChall... 

In [22]:

if run_ichallenge_glaucoma:
    images_path = r"C:\Users\Prinzessin\projects\image_data\iChallenge_Glaucoma_OD_Fovea\images_Glaucoma"
    masks_path  = r"C:\Users\Prinzessin\projects\image_data\iChallenge_Glaucoma_OD_Fovea\Disc_Cup_Masks\Glaucoma"
    new_masks_path  = r"C:\Users\Prinzessin\projects\image_data\iChallenge_Glaucoma_OD_Fovea\Disc_Masks_bin"
    if not os.path.exists(new_masks_path):
        os.makedirs(new_masks_path)
    csv_path    = r"mt_data_ichallenge_glaucoma.csv"
    weight = 0.1
    generate_semisupseg_csv_file(images_path, masks_path, csv_path, weight, image_postfix=".jpg", mask_postfix=".bmp", split_percentage=90, new_mask_path=new_masks_path)

                                                img_path  \
g0006  C:\Users\Prinzessin\projects\image_data\iChall...   
g0001  C:\Users\Prinzessin\projects\image_data\iChall...   
g0034  C:\Users\Prinzessin\projects\image_data\iChall...   
g0002  C:\Users\Prinzessin\projects\image_data\iChall...   
g0036  C:\Users\Prinzessin\projects\image_data\iChall...   
g0014  C:\Users\Prinzessin\projects\image_data\iChall...   
g0020  C:\Users\Prinzessin\projects\image_data\iChall...   
g0027  C:\Users\Prinzessin\projects\image_data\iChall...   
g0005  C:\Users\Prinzessin\projects\image_data\iChall...   
g0017  C:\Users\Prinzessin\projects\image_data\iChall...   
g0008  C:\Users\Prinzessin\projects\image_data\iChall...   
g0007  C:\Users\Prinzessin\projects\image_data\iChall...   
g0021  C:\Users\Prinzessin\projects\image_data\iChall...   
g0038  C:\Users\Prinzessin\projects\image_data\iChall...   
g0030  C:\Users\Prinzessin\projects\image_data\iChall...   
g0032  C:\Users\Prinzessin\projects\imag

In [23]:
if run_ichallenge_non_amd:
    images_path = r"C:\Users\Prinzessin\projects\image_data\iChallenge_AMD_OD_Fovea_lesions\images_AMD"
    masks_path  = r"C:\Users\Prinzessin\projects\image_data\iChallenge_AMD_OD_Fovea_lesions\Disc_Masks"
    new_masks_path = r"C:\Users\Prinzessin\projects\image_data\iChallenge_AMD_OD_Fovea_lesions\Disc_Masks_bin"
    if not os.path.exists(new_masks_path):
        os.makedirs(new_masks_path)
    csv_path    = r"mt_data_ichallenge_non_amd.csv"
    weight = 0.1
    generate_semisupseg_csv_file(images_path, masks_path, csv_path, weight, image_postfix=".jpg", mask_postfix=".bmp", split_percentage=90, new_mask_path=new_masks_path)

                                                img_path  \
A0068  C:\Users\Prinzessin\projects\image_data\iChall...   
A0006  C:\Users\Prinzessin\projects\image_data\iChall...   
A0080  C:\Users\Prinzessin\projects\image_data\iChall...   
A0025  C:\Users\Prinzessin\projects\image_data\iChall...   
A0008  C:\Users\Prinzessin\projects\image_data\iChall...   
...                                                  ...   
A0030  C:\Users\Prinzessin\projects\image_data\iChall...   
A0081  C:\Users\Prinzessin\projects\image_data\iChall...   
A0034  C:\Users\Prinzessin\projects\image_data\iChall...   
A0004  C:\Users\Prinzessin\projects\image_data\iChall...   
A0087  C:\Users\Prinzessin\projects\image_data\iChall...   

                                                msk_path   mode  weight  
A0068  C:\Users\Prinzessin\projects\image_data\iChall...  train     0.1  
A0006  C:\Users\Prinzessin\projects\image_data\iChall...  train     0.1  
A0080  C:\Users\Prinzessin\projects\image_data\iChall... 

In [24]:
if run_ichallenge_unlabelled:
    images_path = r"C:\Users\Prinzessin\projects\image_data\iChallenge_AMD_OD_Fovea_lesions\ADAM_400_testset"
    masks_path  = None
    csv_path    = "mt_data_ichallenge_unlabelled.csv"
    weight = 0
    generate_semisupseg_csv_file(images_path, masks_path, csv_path, weight, image_postfix=".jpg", mask_postfix=".bmp", split_percentage=100)

                                                img_path mask_path   mode  \
V0300  C:\Users\Prinzessin\projects\image_data\iChall...      None  train   
V0260  C:\Users\Prinzessin\projects\image_data\iChall...      None  train   
V0076  C:\Users\Prinzessin\projects\image_data\iChall...      None  train   
V0287  C:\Users\Prinzessin\projects\image_data\iChall...      None  train   
V0281  C:\Users\Prinzessin\projects\image_data\iChall...      None  train   
...                                                  ...       ...    ...   
V0182  C:\Users\Prinzessin\projects\image_data\iChall...      None  train   
V0192  C:\Users\Prinzessin\projects\image_data\iChall...      None  train   
V0210  C:\Users\Prinzessin\projects\image_data\iChall...      None  train   
V0084  C:\Users\Prinzessin\projects\image_data\iChall...      None  train   
V0064  C:\Users\Prinzessin\projects\image_data\iChall...      None  train   

       weight  
V0300       0  
V0260       0  
V0076       0  
V0287      